In [2]:
pip install psycopg2

  Using cached psycopg2-2.9.10-cp312-cp312-win_amd64.whl.metadata (5.0 kB)
Using cached psycopg2-2.9.10-cp312-cp312-win_amd64.whl (1.2 MB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import psycopg2

In [ ]:
conn = psycopg2.connect(
    dbname= DB_NAME,
    user= DB_USER,
    password= DB_PASWORD,
    host= DB_HOST,
    port=DB_PORT
)

In [5]:
cur = conn.cursor()

In [6]:
cur.execute("SELECT version();")
# Fetch and print the result
print(cur.fetchall())

[('PostgreSQL 16.4, compiled by Visual C++ build 1940, 64-bit',)]


In [ ]:
# Rollback any previous transaction that might have failed
conn.rollback()

query1 = """
    CREATE TABLE ciudad (
    id_ciudad SERIAL PRIMARY KEY,
    nombre_ciudad TEXT
    );
"""
cur.execute(query1)
conn.commit()  # Commit the transaction to save changes

In [13]:
conn.rollback()
query2 = """CREATE TABLE eventos (
    id_evento SERIAL PRIMARY KEY,
    nombre_evento TEXT,
    url_evento TEXT,
    codigo_postal INT,
    direccion TEXT,
    horario TEXT,
    fecha_inicio DATE,
    fecha_fin DATE,
    organizacion TEXT,
    id_ciudad INT REFERENCES ciudad(id_ciudad) ON DELETE CASCADE
);
CREATE TABLE hoteles (
    id_hotel SERIAL PRIMARY KEY, -- Puede ser VARCHAR(5)
    nombre_hotel TEXT,
    competencia BOOL,
    valoracion FLOAT CHECK (valoracion BETWEEN 1 AND 5),
    id_ciudad INT REFERENCES ciudad(id_ciudad) ON DELETE CASCADE
);
CREATE TABLE clientes (
    id_cliente VARCHAR(50) PRIMARY KEY,
    nombre TEXT,
    apellido TEXT,
    mail TEXT UNIQUE CHECK (mail LIKE '%@%')
);
CREATE TABLE reservas (
    id_reserva VARCHAR(50) PRIMARY KEY,
    fecha_reserva DATE,
    inicio_estancia DATE,
    final_estancia DATE,
    precio_noche FLOAT CHECK (precio_noche >= 0),
    id_cliente VARCHAR(50) REFERENCES clientes(id_cliente) ON DELETE CASCADE,
    id_hotel INT REFERENCES hoteles(id_hotel) ON DELETE CASCADE
    
);"""

cur.execute(query2)
conn.commit()  # Commit the transaction to save changes

In [142]:
#importamos los datos que insertaremos en la base de datos
df_reservas = pd.read_pickle("../data/reservas_hoteles_final.pkl")
df_eventos = pd.read_pickle("../data/eventos_relevantes.pkl")

# para la tabla de eventos agregamos un id de evento serial 
df_eventos["id_evento"] = np.arange(1, len(df_eventos) + 1)
df_reservas.info(),df_eventos.info()


<class 'pandas.core.frame.DataFrame'>
Index: 15023 entries, 0 to 15097
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id_hotel         15023 non-null  int64         
 1   nombre_hotel     15023 non-null  object        
 2   ciudad           15023 non-null  object        
 3   estrellas        15023 non-null  float64       
 4   precio_noche     15023 non-null  float64       
 5   fecha_reserva    15023 non-null  datetime64[ns]
 6   inicio_estancia  15023 non-null  datetime64[ns]
 7   final_estancia   15023 non-null  datetime64[ns]
 8   competencia      15023 non-null  bool          
 9   id_reserva       15023 non-null  object        
 10  id_cliente       15023 non-null  object        
 11  nombre           15023 non-null  object        
 12  apellido         15023 non-null  object        
 13  mail             15023 non-null  object        
dtypes: bool(1), datetime64[ns](3), float64(2), 

(None, None)

In [26]:
conn.rollback()
# insertamos los datos en la tabla de ciudad 
cur.execute("INSERT INTO ciudad (nombre_ciudad) VALUES ('Madrid')")
conn.commit()

In [68]:
conn.rollback()

data_eventos = []
for _, row in df_eventos.iterrows():
    id_evento = row["id_evento"]
    nombre_evento = row["nombre_evento"]
    url_evento = row["url_evento"]
    codigo_postal = int(row["codigo_postal"]) if pd.notna(row["codigo_postal"]) else None
    direccion = row["direccion"]
    horario = row["horario"]
    fecha_inicio = pd.to_datetime(row["inicio_evento"])
    fecha_fin = pd.to_datetime(row["fin_evento"])
    organizacion = row["organizacion"]
    ciudad = row["ciudad"]
    id_ciudad = 2

    # Mueve el append dentro del bucle
    data_eventos.append([id_evento, nombre_evento, url_evento, codigo_postal, direccion, horario, fecha_inicio, fecha_fin, organizacion, id_ciudad]) 

# Verifica que la lista no esté vacía
if not data_eventos:
    raise ValueError("La lista data_eventos está vacía. Verifica df_eventos.")

# Crear DataFrame
tabla_eventos = pd.DataFrame(data_eventos, columns=["id_evento", "nombre_evento", "url_evento", "codigo_postal", "direccion", "horario", "inicio_evento", "fin_evento", "organizacion", "id_ciudad"])
print(tabla_eventos.head())  # Para depuración

# Query de inserción
insert_query = """
INSERT INTO eventos (id_evento, nombre_evento, url_evento, codigo_postal, direccion, horario, fecha_inicio, fecha_fin, organizacion, id_ciudad)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
ON CONFLICT (id_evento) DO NOTHING;
                """

# Verifica que haya datos antes de la inserción
if data_eventos:
    cur.executemany(insert_query, data_eventos)
    conn.commit()
else:
    print("No hay datos para insertar en la tabla eventos.")


   id_evento                                 nombre_evento  \
0          1       25º aniversario de la revista La Fragua   
1          2  60 Premio Reina Sofía de Pintura y Escultura   
2          3                                   A toda vela   
3          4     Acompañamiento digital a personas mayores   
4          5                       Acompañar en la pérdida   

                                          url_evento  codigo_postal  \
0  http://www.madrid.es/sites/v/index.jsp?vgnextc...        28005.0   
1  http://www.madrid.es/sites/v/index.jsp?vgnextc...        28009.0   
2  http://www.madrid.es/sites/v/index.jsp?vgnextc...        28045.0   
3  http://www.madrid.es/sites/v/index.jsp?vgnextc...            NaN   
4  http://www.madrid.es/sites/v/index.jsp?vgnextc...        28029.0   

                      direccion horario inicio_evento fin_evento  \
0             CALLE SAN JUSTO 5            2025-02-24 2025-03-02   
1              PASEO COLOMBIA 1            2025-02-27 2025-03-23

# Data frame hoteles y tabla SQL hoteles

Para inserta la informacion de hoteles, hay que hacer group by de los hoteles que tenemos en la tabla de reservas

In [97]:
#separamos el df de hoteles propios para hacer el average de valoracion que tienen
df_propios = df_reservas[df_reservas['competencia'] == False]

df_propios_agrupados = df_propios.groupby(['id_hotel','nombre_hotel', 'competencia']).agg({'estrellas':'mean'}).round(2).reset_index()
df_propios_agrupados

,id_hotel,nombre_hotel,competencia,estrellas
0,1,Gran Hotel Madrid,False,3.06
1,2,Hotel Brisas del Mar,False,3.09
2,3,Hotel Camino del Sol,False,3.08
3,4,Hotel Costa Azul,False,3.11
4,5,Hotel Encanto Real,False,3.02
5,6,Hotel Jardines del Rey,False,2.95
6,7,Hotel Las Estrellas,False,2.91
7,8,Hotel Los Almendros,False,3.01
8,9,Hotel Luz de Madrid,False,3.05
9,10,Hotel Maravilla Real,False,2.97


In [99]:
df_competencia = df_reservas[df_reservas['competencia'] == True]
df_competencia_agrupados = df_competencia.groupby(['id_hotel','nombre_hotel', 'competencia']).agg({'estrellas':'mean'}).round(2).reset_index()
df_competencia_agrupados

,id_hotel,nombre_hotel,competencia,estrellas
0,103,Novotel Madrid Center,True,4.6
1,113,ibis Styles Madrid Prado,True,4.7
2,114,ibis Styles Madrid Centro Maravillas,True,4.7
3,117,ibis Styles Madrid City Las Ventas,True,4.6
4,128,ibis Madrid Centro las Ventas,True,4.5
5,131,ibis budget Madrid Centro Lavapies,True,4.3
6,135,Novotel Madrid City Las Ventas,True,4.6
7,181,ibis budget Madrid Calle 30,True,4.4
8,186,ibis budget Madrid Centro las Ventas,True,4.3
9,194,Mercure Madrid Centro,True,4.4


In [101]:
# unificamos los dos dataframes
df_hoteles = pd.concat([df_propios_agrupados, df_competencia_agrupados])
df_hoteles

# ahora cambiamos el nombre de la columna estrella por valoracion
df_hoteles.rename(columns = {'estrellas':'valoracion'}, inplace = True)
df_hoteles

,id_hotel,nombre_hotel,competencia,valoracion
0,1,Gran Hotel Madrid,False,3.06
1,2,Hotel Brisas del Mar,False,3.09
2,3,Hotel Camino del Sol,False,3.08
3,4,Hotel Costa Azul,False,3.11
4,5,Hotel Encanto Real,False,3.02
5,6,Hotel Jardines del Rey,False,2.95
6,7,Hotel Las Estrellas,False,2.91
7,8,Hotel Los Almendros,False,3.01
8,9,Hotel Luz de Madrid,False,3.05
9,10,Hotel Maravilla Real,False,2.97


In [103]:
conn.rollback()

data_hoteles = []
for _, row in df_hoteles.iterrows():
    id_hotel = row["id_hotel"]
    nombre_hotel = row["nombre_hotel"]
    competencia = row["competencia"]
    valoracion = float(row["valoracion"]) if pd.notna(row["valoracion"]) else None

    # Mueve el append dentro del bucle
    data_hoteles.append([id_hotel, nombre_hotel, competencia, valoracion]) 

# Verifica que la lista no esté vacía
if not data_hoteles:
    raise ValueError("La lista data_eventos está vacía. Verifica df_eventos.")

# Crear DataFrame
tabla_hoteles = pd.DataFrame(data_hoteles, columns=["id_hotel", "nombre_hotel", "competencia", "valoracion"])
print(tabla_hoteles)  # Para depuración



    id_hotel                          nombre_hotel  competencia  valoracion
0          1                     Gran Hotel Madrid        False        3.06
1          2                  Hotel Brisas del Mar        False        3.09
2          3                  Hotel Camino del Sol        False        3.08
3          4                      Hotel Costa Azul        False        3.11
4          5                    Hotel Encanto Real        False        3.02
5          6                Hotel Jardines del Rey        False        2.95
6          7                   Hotel Las Estrellas        False        2.91
7          8                   Hotel Los Almendros        False        3.01
8          9                   Hotel Luz de Madrid        False        3.05
9         10                  Hotel Maravilla Real        False        2.97
10        11                    Hotel Mirador Real        False        2.98
11        12                     Hotel Monte Verde        False        3.10
12        13

In [ ]:
conn.rollback()

# Query de inserción
insert_query = """
INSERT INTO hoteles (id_hotel, nombre_hotel, competencia, valoracion)
VALUES (%s, %s, %s, %s);
                """
# Verifica que haya datos antes de la inserción
if data_hoteles:
    cur.executemany(insert_query, data_hoteles)
    conn.commit()
else:
    print("No hay datos para insertar en la tabla eventos.")


In [110]:
conn.rollback()
# agregamos query de id_ciudad para la tabla de hoteles
query = """
UPDATE hoteles
SET id_ciudad = 2;
"""
cur.execute(query)
conn.commit()


# Tabla de clientes

In [155]:
df_clientes = df_reservas[['id_cliente','nombre','apellido','mail']]

#eliminamos los duplicado de "id_cliente" y "mail"
df_clientes_limpio = df_clientes.drop_duplicates(subset=['id_cliente','mail'])
df_clientes_limpio

,id_cliente,nombre,apellido,mail
0,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Feliciana,Cantón,feliciana.cantón@example.com
2,6ec4d7d2-e382-47cc-94f6-c647577d4035,Maite,Calatayud,maite.calatayud@example.com
3,aed17fe8-eb18-4f69-bf08-df1afdc339c3,Tecla,Bonet,tecla.bonet@example.com
4,710a2fbb-264f-426a-aa66-f8fffd146a2d,Amílcar,Andrés,amílcar.andrés@example.com
5,df42ae3d-0b9b-4c06-b22e-939323833ef8,Joan,Vazquez,joan.vazquez@example.com
...,...,...,...,...
14995,de729428-ec23-4665-a5c9-79e3ec282470,Lorena,Román,lorena.román@example.com
14996,8469b819-d96a-4904-9cdd-710764f6301f,Toño,Narváez,toño.narváez@example.com
14997,e89751f4-3621-4636-81d3-90e2baa709cb,Nadia,Verdugo,nadia.verdugo@example.com
14998,c4117358-fd89-44cd-b388-4ec6d3538c72,Custodia,Marco,custodia.marco@example.com


In [156]:
#instanciamos para subir a base de datos
data_clientes = []
for _, row in df_clientes_limpio.iterrows():
    id_cliente = row["id_cliente"]
    nombre = row["nombre"]
    apellido = row["apellido"]
    mail = row["mail"]

    # Mueve el append dentro del bucle
    data_clientes.append([id_cliente, nombre, apellido, mail])

In [157]:
conn.rollback()

 # Query de inserción
insert_query = """
 INSERT INTO clientes (id_cliente, nombre, apellido, mail)
 VALUES (%s, %s, %s, %s);
                """
# Verifica que haya datos antes de la inserción
if data_clientes:
    cur.executemany(insert_query, data_clientes)
    conn.commit()
else:
    print("No hay datos para insertar en la tabla eventos.")


# Tabla de reservas

In [158]:
# ahora comenzamos a ordenar los datos para subir las reservas en los hoteles
df_reservas.head()

# agrupamos por reservas
df_reservas_agrupadas = df_reservas.groupby(['id_reserva','fecha_reserva','inicio_estancia','final_estancia','precio_noche','id_cliente','id_hotel']).agg({'competencia':'first'}).reset_index()
df_reservas_agrupadas

,id_reserva,fecha_reserva,inicio_estancia,final_estancia,precio_noche,id_cliente,id_hotel,competencia
0,00024263-f20c-4c90-ac15-1a64be97811f,2025-02-09,2025-03-01,2025-03-02,182.97,ad57a54e-f788-44b1-a5ea-3438d45dd0ee,18,False
1,0007490e-3c9f-4ba4-9440-85670deb2c9b,2025-02-04,2025-03-01,2025-03-02,415.42,73312e4b-1129-49d5-bf80-60c24b72f192,9,False
2,00094a12-93cd-479e-8799-b05fc5a4d2b6,2025-02-05,2025-03-01,2025-03-02,135.78,13d3776c-62eb-4ca2-a491-d6b0774e0b65,5,False
3,000e5ed2-5639-43a5-8bfe-bd3c39156c9a,2025-02-07,2025-03-01,2025-03-02,298.08,2a448cc9-8df1-4e2c-86e4-c69adf34a273,11,False
4,000eecd9-8b82-468f-bbbb-1acffb643f35,2025-02-05,2025-03-01,2025-03-02,136.77,2024e846-3cf6-44fb-aed4-00481053f6b9,16,False
...,...,...,...,...,...,...,...,...
14920,ffecadea-62d2-49ef-9472-f8c3229bac83,2025-02-05,2025-03-01,2025-03-02,416.45,39dcc3d6-3d91-46f5-8572-e745af8f76b2,13,False
14921,ffecbd34-cbd2-48f5-b02b-a2790a2feb24,2025-02-07,2025-03-01,2025-03-02,406.48,0f6c55f8-f121-477b-8eec-bc6655c9fcdb,12,False
14922,fff0229e-0cef-44d3-928e-ee8928ae9840,2025-02-03,2025-03-01,2025-03-02,94.20,d0118ca7-1df4-4d8e-9c14-c53009236e2b,4,False
14923,fff5c27d-b93d-45e5-9a86-78ac30c1358e,2025-02-20,2025-03-01,2025-03-02,97.00,72216a57-90ba-4ade-b0b0-fc7220cf2b58,186,True


In [159]:
# instanciamos para subir a la base de datos
data_reservas = []
for _, row in df_reservas_agrupadas.iterrows():
    id_reserva = row["id_reserva"]
    fecha_reserva = pd.to_datetime(row["fecha_reserva"])
    inicio_estancia = pd.to_datetime(row["inicio_estancia"])
    final_estancia = pd.to_datetime(row["final_estancia"])
    precio_noche = float(row["precio_noche"]) if pd.notna(row["precio_noche"]) else None
    id_cliente = row["id_cliente"]
    id_hotel = row["id_hotel"]

    # Mueve el append dentro del bucle
    data_reservas.append([id_reserva, fecha_reserva, inicio_estancia, final_estancia, precio_noche, id_cliente, id_hotel])

In [160]:
conn.rollback()

# Query de inserción
insert_query = """
INSERT INTO reservas (id_reserva, fecha_reserva, inicio_estancia, final_estancia, precio_noche, id_cliente, id_hotel)
VALUES (%s, %s, %s, %s, %s, %s, %s);
                """
# Verifica que haya datos antes de la inserción
if data_reservas:
    cur.executemany(insert_query, data_reservas)
    conn.commit()
else:
    print("No hay datos para insertar en la tabla eventos.")

In [ ]:
cur.close()
conn.close()